In [61]:
import opentrons.execute
from opentrons import types

In [2]:
metadata = {
    'protocolName': 'Nanopore LSK109(1) DNA endprep_flongle',
    'author': 'Seong-Kun Bak <tjdrns27@kribb.re.kr>',
    'apiLevel': '2.11',
    'description': 'with movable PCR tube pause the protocol when the tube should move'
}

In [3]:
protocol = opentrons.execute.get_protocol_api("2.11")

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


In [ ]:
protocol.home()

## Labwares

In [4]:
module_thermocycler = protocol.load_module("thermocycler")
module_magnetic = protocol.load_module('magnetic module gen2', '4')

In [5]:
import json
with open("./labware/bioneer_96_tuberack_200ul.json") as labware_file:
    bioneer_tube = json.load(labware_file)
with open("./labware/biorad_96_tuberack_200ul.json") as labware_file:
    biorad_tube = json.load(labware_file)

In [6]:
enzyme_rack = protocol.load_labware("opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap", 1)
falcon_rack = protocol.load_labware("opentrons_6_tuberack_falcon_50ml_conical", 2)
dna_plate = protocol.load_labware_from_definition(bioneer_tube, 5)
mag_plate = module_magnetic.load_labware("biorad_96_tuberack_200ul", 4)
trash = protocol.loaded_labwares[12]["A1"]

tiprack_20_sin = protocol.load_labware("opentrons_96_tiprack_20ul", 3)
tiprack_300_sin = protocol.load_labware("opentrons_96_tiprack_300ul", 6)

This labware ({}) is not explicitly compatible with the Magnetic Module. You will have to specify a height when calling engage().


In [7]:
p20_sin = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tiprack_20_sin])
p300_sin = protocol.load_instrument("p300_single_gen2", "left", tip_racks=[tiprack_300_sin])

In [8]:
## Sample Wells
start_position = 0 ## start from A1
sample_number = 2

In [9]:
dna_well = [dna_plate.wells()[i] for i in range(start_position, start_position + sample_number)]
final_well = [dna_plate.wells()[i] for i in range(start_position + 16, start_position + 16 + sample_number)]
mag_well = [mag_plate.wells()[i] for i in range(start_position, start_position + sample_number)]

## Reagents

In [10]:
repair_buf = enzyme_rack["A1"]
repair_enzyme = enzyme_rack["A2"]

endprep_buf = enzyme_rack["A3"]
endprep_enzyme = enzyme_rack["A4"]

water = enzyme_rack["A5"]

In [11]:
ampure_beads = falcon_rack["A1"]
ethanol_70 = falcon_rack["A2"]

In [12]:
## Start Tiprack positions
p20_sin.starting_tip = tiprack_20_sin.well("C2")
p300_sin.starting_tip = tiprack_300_sin.well("E1")

In [62]:
## Functions
def remove_supernantant(pipette, volume, src, dest):
    pipette.pick_up_tip()
    pipette.flow_rate.aspirate=20
    pipette.aspirate(volume, src.bottom().move(types.Point(x=0, y=0, z=0)))
    pipette.dispense(volume, dest)
    pipette.drop_tip()

In [80]:
## Enzyme reaction
p20_sin.flow_rate.aspirate=5
p20_sin.flow_rate.dispense=5
p20_sin.transfer(3.5, repair_buf, dna_well, rate=0.5, new_tip="always")
p20_sin.transfer(2, repair_enzyme, dna_well, touch_tip = True, rate=0.3, new_tip="always", mix_after=(2,15))
p20_sin.transfer(3.5, endprep_buf, dna_well, new_tip="always")
p20_sin.transfer(3, endprep_enzyme, dna_well, touch_tip=True, new_tip="always", mix_after=(2, 15))

<InstrumentContext: p20_single_v2.2 in RIGHT>

In [ ]:
module_thermocycler.open_lid()

In [ ]:
protocol.pause("1. Check enzyme\n2. Spindown\n3. Move sample tube to thermocycler\n4. Resume")
protocol.resume()

In [17]:
module_thermocycler.close_lid()

alarm/error outside hard halt: ALARM: Hard limit +Z
alarm/error: command=M907 A0.1 B0.3 C0.3 X0.3 Y0.3 Z0.8 G4 P0.005 G0 Z208.0 

, resp=ALARM: Hard limit +Z


<ThermocyclerLidStatus.CLOSED: 'closed'>

In [18]:
profile = [{'temperature': 20, 'hold_time_minutes': 5},
            {'temperature': 65, 'hold_time_minutes': 5},
            {'temperature': 4, 'hold_time_minutes': 2}]
module_thermocycler.execute_profile(steps=profile, repetitions=1, block_max_volume=60)

In [19]:
module_thermocycler.deactivate()
module_thermocycler.open_lid()

In [20]:
module_thermocycler.open_lid()

<ThermocyclerLidStatus.OPEN: 'open'>

In [21]:
p300_sin.pick_up_tip()
p300_sin.mix(5, 300, ampure_beads.bottom(z=3))
p300_sin.touch_tip()
p300_sin.drop_tip()

<InstrumentContext: p300_single_v2.1 in LEFT>

In [29]:
# 옮기다 bead가 떨어지는 문제발생
p300_sin.transfer(80, ampure_beads.bottom(z=15), dna_well, mix_after=(3, 30), touch_tip = True, rate=0.3, new_tip="always")

<InstrumentContext: p300_single_v2.1 in LEFT>

In [15]:
module_magnetic.engage(height_from_base=2.5)


In [ ]:
protocol.delay(minutes=5)

In [91]:
## Functions
def remove_supernantant(pipette, volume, src, dest):
    pipette.pick_up_tip()
    pipette.flow_rate.aspirate=20
    pipette.aspirate(volume, src.bottom().move(types.Point(x=0, y=0, z=-4.2)))
    pipette.dispense(volume, dest)
    pipette.drop_tip()

In [96]:
for src in mag_well:
    remove_supernantant(p300_sin, 200, src, trash)

In [97]:
p300_sin.flow_rate.aspirate=100
p300_sin.flow_rate.dispense=50
for _ in range(1):
    ## Ethanol
    p300_sin.pick_up_tip()
    for src in mag_well:
        p300_sin.flow_rate.aspirate=100
        p300_sin.flow_rate.dispense=50
        p300_sin.aspirate(190, ethanol_70)
        p300_sin.dispense(190, src.top(-3))
    p300_sin.drop_tip()
    ## Remove
    for src in mag_well:
        remove_supernantant(p300_sin, 200, src, trash)

In [98]:
module_magnetic.disengage()